# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import trange
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [2]:
geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')

datasets = [geo_data_0, geo_data_1, geo_data_2]

In [3]:
def view(data):
    print('Обзор данных', '\n')
    print(data.sample(10))
    print('\n')
    print('Информация о формате, количестве строк и пропусках', '\n')
    print(data.info())
    print('\n')
    print('Описание (статистика)', '\n')
    print(data.describe())
    print('\n')
    print('Дубликаты', '\n')
    print(data.duplicated().sum())
    print('-' * 65)
    print('\n' * 3)

In [4]:
i=0
for data in datasets:
    if i==0:
        data.name='geo_data_0'
    elif i==1:
        data.name='geo_data_1'
    else:
        data.name='geo_data_2'
    print('Регион ', data.name, '\n') 
    view(data)
    i+=1

Регион  geo_data_0 

Обзор данных 

          id        f0        f1        f2     product
45764  UZ8RA  1.063835  0.250076  4.842919   96.890407
93521  NNX2q  1.720158  0.051122  5.020973  168.008540
4726   mhepZ  1.066173  0.131666 -0.119478   21.116589
18257  DJrm0  1.005093  0.389595  2.377363   77.064842
26264  L2PSJ  0.801502  0.596961  3.403106   32.662378
54302  qLYEX  0.587974 -0.511534  1.094287   59.603058
70843  TL6vR -0.086625  0.433608  4.237878  103.023616
25676  VLTwg -0.196967  0.887959 -1.957792   28.628293
57574  XJrez  1.040517  0.229426 -3.314861   68.813397
47939  5F6vZ  0.838614  0.371312  1.588534   78.020927


Информация о формате, количестве строк и пропусках 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64


Пропусков нет, дубликатов нет, форматы колонок соответствуют значениям.

Проверим зависимость между признаками по регионам

In [5]:
for data in datasets:
    print(data.corr(), '\n')

               f0        f1        f2   product
f0       1.000000 -0.440723 -0.003153  0.143536
f1      -0.440723  1.000000  0.001724 -0.192356
f2      -0.003153  0.001724  1.000000  0.483663
product  0.143536 -0.192356  0.483663  1.000000 

               f0        f1        f2   product
f0       1.000000  0.182287 -0.001777 -0.030491
f1       0.182287  1.000000 -0.002595 -0.010155
f2      -0.001777 -0.002595  1.000000  0.999397
product -0.030491 -0.010155  0.999397  1.000000 

               f0        f1        f2   product
f0       1.000000  0.000528 -0.000448 -0.001987
f1       0.000528  1.000000  0.000779 -0.001012
f2      -0.000448  0.000779  1.000000  0.445871
product -0.001987 -0.001012  0.445871  1.000000 



Во второй таблице (geo_data_1) наблюдается сильная линейная зависимость между признаком f2 и целевым признаком product.

Удалим колонку id ввиду её неинформативности

In [6]:
print(geo_data_0)

          id        f0        f1        f2     product
0      txEyH  0.705745 -0.497823  1.221170  105.280062
1      2acmU  1.334711 -0.340164  4.365080   73.037750
2      409Wp  1.022732  0.151990  1.419926   85.265647
3      iJLyR -0.032172  0.139033  2.978566  168.620776
4      Xdl7t  1.988431  0.155413  4.751769  154.036647
...      ...       ...       ...       ...         ...
99995  DLsed  0.971957  0.370953  6.075346  110.744026
99996  QKivN  1.392429 -0.382606  1.273912  122.346843
99997  3rnvd  1.029585  0.018787 -1.348308   64.375443
99998  7kl59  0.998163 -0.528582  1.583869   74.040764
99999  1CWhH  1.764754 -0.266417  5.722849  149.633246

[100000 rows x 5 columns]


In [7]:
geo_data_0 = geo_data_0.drop(['id'], axis=1)
geo_data_1 = geo_data_1.drop(['id'], axis=1)
geo_data_2 = geo_data_2.drop(['id'], axis=1)

In [8]:
#geo_data_0.head()
#geo_data_1.head()

In [9]:
#for i in range(len(datasets)):
#    print(i)
 #   datasets[i] = datasets[i].drop(columns=['id'], axis=0)
#  print(datasets[i].columns)
    
    #print(datasets[i])

# Вывод

## Обучение и проверка модели

In [10]:
def test_split(df, target):
    features = df.drop([target] , axis=1)
    target = df[target]

    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)
    
    return features_train, features_valid, target_train, target_valid

In [11]:
features_train_0, features_valid_0, target_train_0, target_valid_0 = test_split(geo_data_0, 'product')

In [12]:
features_train_0.head()

,f0,f1,f2
27212,0.022450,0.951034,2.197333
7866,1.766731,0.007835,6.436602
62041,0.724514,0.666063,1.840177
70185,-1.104181,0.255268,2.026156
82230,-0.635263,0.747990,6.643327


Проведем обучение модели

In [13]:
model = LinearRegression()

model.fit(features_train_0, target_train_0)
predicted_target_0 = model.predict(features_valid_0)
print('RMSE -', mean_squared_error(target_valid_0, predicted_target_0)**0.5)

RMSE - 37.5794217150813


Расссмотрим гипотезу на geo_data_1 и geo_data_2:

In [14]:
features_train_1, features_valid_1, target_train_1, target_valid_1 = test_split(geo_data_1, 'product')
model.fit(features_train_1, target_train_1)
predicted_target_1 = model.predict(features_valid_1)
print('RMSE -', mean_squared_error(target_valid_1, predicted_target_1)**0.5)

RMSE - 0.893099286775617


In [15]:
features_train_2, features_valid_2, target_train_2, target_valid_2 = test_split(geo_data_2, 'product')
model.fit(features_train_2, target_train_2)
predicted_target_2 = model.predict(features_valid_2)
print('RMSE -', mean_squared_error(target_valid_2, predicted_target_2)**0.5)

RMSE - 40.02970873393434


Произведем вывод предсказанного запаса с реальным и выведем RMSE:

In [16]:
model = LinearRegression()

def oil_predict(df, target_name):
    features = df.drop([target_name] , axis=1)
    target = df[target_name]
    
    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=5)
    
    model.fit(features_train, target_train)
    predicted_target = model.predict(features_valid)
    data = {'real_product' : target_valid,
            'predicted_product' : predicted_target         
    }
    data_frame = pd.DataFrame(data, columns = ['real_product', 'predicted_product'])
    rmse = mean_squared_error(target_valid, predicted_target)**0.5
    mean = data_frame['predicted_product'].mean()
    oil_sum = data_frame['predicted_product'].sum()
    return data_frame, rmse, mean, oil_sum

In [17]:
geo_data_0_predicted, geo_data_0_rmse, geo_data_0_predicted_mean, geo_data_0_predicted_sum = oil_predict(geo_data_0, 'product')

In [18]:
geo_data_0_predicted.head()

,real_product,predicted_product
60743,119.220564,118.026686
33949,169.808150,110.858985
52805,131.589165,76.589517
11804,144.585224,100.990823
9229,184.075418,90.048812


In [19]:
print('RMSE -', geo_data_0_rmse)
print('Средний запас сырья -', geo_data_0_predicted_mean)

RMSE - 37.80284530541535
Средний запас сырья - 92.62394691552272


In [20]:
geo_data_1_predicted, geo_data_1_rmse, geo_data_1_predicted_mean, geo_data_1_predicted_sum = oil_predict(geo_data_1, 'product')

In [21]:
geo_data_1_predicted.head()

,real_product,predicted_product
60743,80.859783,80.991708
33949,53.906522,53.729323
52805,3.179103,3.751569
11804,107.813044,108.245842
9229,80.859783,82.008314


In [22]:
print('RMSE -', geo_data_1_rmse)
print('Средний запас сырья -', geo_data_1_predicted_mean)

RMSE - 0.8911637870714586
Средний запас сырья - 68.64756489237209


In [23]:
geo_data_2_predicted, geo_data_2_rmse, geo_data_2_predicted_mean, geo_data_2_predicted_sum = oil_predict(geo_data_2, 'product')

In [24]:
geo_data_2_predicted.head()

,real_product,predicted_product
60743,186.714538,88.685519
33949,73.641755,118.919701
52805,125.190694,98.429155
11804,130.826729,114.772869
9229,14.983400,54.375878


In [25]:
print('RMSE -', geo_data_2_rmse)
print('Средний запас сырья -', geo_data_2_predicted_mean)

RMSE - 40.27855914124022
Средний запас сырья - 95.02417827030403


Оценка общего запаса на регион по предсказанным данным:

In [26]:
print('Суммарный предсказанный запас в регионе 0 -', geo_data_0_predicted_sum)
print('Суммарный реальный запас в регионе 0 -', geo_data_0_predicted['real_product'].sum())
print()
print('Суммарный предсказанный запас в регионе 1 -', geo_data_1_predicted_sum)
print('Суммарный реальный запас в регионе 1 -', geo_data_1_predicted['real_product'].sum())
print()
print('Суммарный предсказанный запас в регионе 2 -', geo_data_2_predicted_sum)
print('Суммарный реальный запас в регионе 2 -', geo_data_2_predicted['real_product'].sum())

Суммарный предсказанный запас в регионе 0 - 2315598.672888068
Суммарный реальный запас в регионе 0 - 2322579.7884419193

Суммарный предсказанный запас в регионе 1 - 1716189.1223093022
Суммарный реальный запас в регионе 1 - 1716031.8878765225

Суммарный предсказанный запас в регионе 2 - 2375604.456757601
Суммарный реальный запас в регионе 2 - 2377216.735320481


# Вывод

## Подготовка к расчёту прибыли

Введём переменные по экономической информации:

In [27]:
budget = 1e7 # бюджет всего тыс. рублей
chosen_points = 500
best_points = 200 # вышек всего
price_per_barrel = 450 # цена за баррель рублей
risk = 0.025 # допустимый риск
well_cost = budget / best_points # цена за скважину
min_bulk = well_cost/ (price_per_barrel*1000) # минимальный объем для окупаемости
#print('Цена одной скважины: {:.2f}'.format(well_cost))
#print('Минимальный объем в скважине, для окупаемости: {:.2f} bar'.format(min_bulk))

In [28]:
average_profit_per_point = budget / (best_points)
print('Средняя прибыль на лучшую точку -', average_profit_per_point, 'тыс.руб.')

Средняя прибыль на лучшую точку - 50000.0 тыс.руб.


In [29]:
average_barrels_per_point = average_profit_per_point  / price_per_barrel
print('Средний объем нефти на точку -',average_barrels_per_point, 'тыс. барелей')

Средний объем нефти на точку - 111.11111111111111 тыс. барелей


In [30]:
average_barrels = budget / (price_per_barrel)
print('Средний объем нефти на регион -',average_barrels, 'тыс. барелей')

Средний объем нефти на регион - 22222.222222222223 тыс. барелей


In [31]:
print('Средний запас фактичекой нефти на скважину в регионе 0', geo_data_0['product'].mean())
print('Средний запас фактичекой нефти на скважину в регионе 1', geo_data_1['product'].mean())
print('Средний запас фактичекой нефти на скважину в регионе 2', geo_data_2['product'].mean())

Средний запас фактичекой нефти на скважину в регионе 0 92.50000000000001
Средний запас фактичекой нефти на скважину в регионе 1 68.82500000000002
Средний запас фактичекой нефти на скважину в регионе 2 95.00000000000004


Функция для расчета прибыли на 500 разведанных точек:

In [32]:
def oil_profit_calc(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return price_per_barrel * selected.sum() - budget

# Вывод

## Расчёт прибыли и рисков 

In [33]:
state = np.random.RandomState(12345)
def bootstrap_1000(target, probs):
    values=[]
    lost = 0
    for i in range(1000):
        target_sample = target.sample(replace=True, random_state=state, n=chosen_points)
        probs_sample = probs[target_sample.index]
        profit = oil_profit_calc(target, probs_sample, best_points)
        if profit < 0:
            lost +=1
        values.append(profit)
    
    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    average = values.mean()
    risk = lost / 1000
    return average, lower, upper, risk

In [34]:
average_0, lower_0, upper_0, risk_0 = bootstrap_1000(geo_data_0_predicted['real_product'], 
                                                     geo_data_0_predicted['predicted_product'])

In [35]:
print('Средняя прибыль по региону = {:.2f} тыс.руб.'.format(average_0))
print('95% доверительный интервал от {:.2f} до {:.2f} тыс.руб.'.format(lower_0, upper_0))
print('Процент риска {:.2%}'.format(risk_0))

Средняя прибыль по региону = 459923.07 тыс.руб.
95% доверительный интервал от -44499.97 до 976934.96 тыс.руб.
Процент риска 4.00%


In [36]:
average_1, lower_1, upper_1, risk_1 = bootstrap_1000(geo_data_1_predicted['real_product'], 
                                                     geo_data_1_predicted['predicted_product'])

In [37]:
print('Средняя прибыль по региону = {:.2f} тыс.руб.'.format(average_1))
print('95% доверительный интервал от {:.2f} до {:.2f} тыс.руб.'.format(lower_1, upper_1))
print('Процент риска {:.2%}'.format(risk_1))

Средняя прибыль по региону = 411514.81 тыс.руб.
95% доверительный интервал от 28098.12 до 802992.98 тыс.руб.
Процент риска 2.10%


In [38]:
average_2, lower_2, upper_2, risk_2= bootstrap_1000(geo_data_2_predicted['real_product'], 
                                                     geo_data_2_predicted['predicted_product'])

In [39]:
print('Средняя прибыль по региону = {:.2f} тыс.руб.'.format(average_2))
print('95% доверительный интервал от {:.2f} до {:.2f} тыс.руб.'.format(lower_2, upper_2))
print('Процент риска {:.2%}'.format(risk_2))

Средняя прибыль по региону = 323716.54 тыс.руб.
95% доверительный интервал от -202834.13 до 852197.70 тыс.руб.
Процент риска 11.00%


# Вывод

**Данные по первому (geo_data_0) региону:**

**Данные по второму (geo_data_1) региону:**

**Данные по третьему (geo_data_2) региону:**

# Общий вывод

По каждому региону данные обработаны, рассчитаны средняя прибыль, 95% доверительный интервал и процент риска.

Отсеиваем третий (geo_data_2) регион, так как у этого региона наибольший процент риска - 11.0%

По полученным результатам определяем второй регион (geo_data_2), 
наиболее удовлетворяющий поставленным условиям отбора.

**Данные по второму (geo_data_1) региону:**

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [х]  Весь код выполняется без ошибок
- [х]  Ячейки с кодом расположены в порядке исполнения
- [х]  Выполнен шаг 1: данные подготовлены
- [х]  Выполнен шаг 2: модели обучены и проверены
    - [х]  Данные корректно разбиты на обучающую и валидационную выборки
    - [х]  Модели обучены, предсказания сделаны
    - [х]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [х]  На экране напечатаны результаты
    - [х]  Сделаны выводы
- [х]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [х]  Для всех ключевых значений созданы константы Python
    - [х]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [х]  По предыдущему пункту сделаны выводы
    - [х]  Написана функция расчёта прибыли
- [х]  Выполнен шаг 4: посчитаны риски и прибыль
    - [х]  Проведена процедура *Bootstrap*
    - [х]  Все параметры бутстрепа соответствуют условию
    - [х]  Найдены все нужные величины
    - [х]  Предложен регион для разработки месторождения
    - [х]   Выбор региона обоснован